In [1]:
import pickle
import numpy as np
import random
import pandas as pd

In [2]:
with open('./title_apple_vec.np', 'rb') as f:
    X = pickle.load(f)
X.shape

FileNotFoundError: [Errno 2] No such file or directory: './title_apple_vec.np'

In [ ]:
T = 10          #rounds

In [ ]:
#generate sorted random indices list with length d, where d is randomly picked between D/2 and D-1
def generate_indicies(D):
    d = random.randint(D//2, D-2)
    indicies = random.sample(range(0, D - 1), d)
    return sorted(indicies)

In [ ]:
#generate corped X
def generate_corped_x(X):
    D = len(X[0])
    indices = generate_indicies(D)
    return X[:, indices]

In [ ]:
new_X = generate_corped_x(X)
new_X.shape

In [ ]:
# from sklearn.ensemble import IsolationForest
# from sklearn.neighbors import LocalOutlierFactor
from pyod.models.iforest import IForest
from pyod.models.lof import LOF
from pyod.models.abod import ABOD
from pyod.models.knn import KNN
from tqdm.notebook import tqdm

In [ ]:
AS_iforest = []
PR_iforest = []
for t in tqdm(range(T)):
    X_corped = generate_corped_x(X)
#     clf_iforest = IsolationForest().fit_predict(X_corped)
#     Anomaly_score = clf_iforest.score_samples(X_corped)
    clf_iforest = IForest().fit(X_corped)
    Anomaly_score = clf_iforest.decision_function(X_corped)
    Anomaly_proba = clf_iforest.predict_proba(X_corped)
    AS_iforest.append(Anomaly_score)
    PR_iforest.append(Anomaly_proba)
AS_iforest = np.array(AS_iforest)
PR_iforest = np.array(PR_iforest)
iforest_res = np.mean(AS_iforest, axis=0)
iforest_prob = np.mean(PR_iforest, axis=0)

In [ ]:
AS_LOF = []
PR_LOF = []
for t in tqdm(range(T)):
    X_corped = generate_corped_x(X)
#     clf_LOF = LocalOutlierFactor(n_neighbors=5, contamination=0.1).fit(X_corped)
#     Anomaly_score = clf_LOF.negative_outlier_factor_
    clf_LOF = LOF(n_neighbors=5).fit(X_corped)
    Anomaly_score = clf_LOF.decision_function(X_corped)
    Anomaly_proba = clf_LOF.predict_proba(X_corped)
    AS_LOF.append(Anomaly_score)
    PR_LOF.append(Anomaly_proba)
AS_LOF = np.array(AS_LOF)
PR_LOF = np.array(PR_LOF)
LOF_res = np.mean(AS_LOF, axis=0)
LOF_prob = np.mean(PR_LOF, axis=0)

In [ ]:
AS_ABOD = []
PR_ABOD = []
for t in tqdm(range(T)):
    X_corped = generate_corped_x(X)
    clf_ABOD = ABOD(contamination=0.1, n_neighbors=5, method='fast').fit(X_corped)
    Anomaly_score = clf_ABOD.decision_scores_
    Anomaly_proba = clf_ABOD.predict_proba(X_corped, method='unify')
    AS_ABOD.append(Anomaly_score)
    PR_ABOD.append(Anomaly_proba)
AS_ABOD = np.array(AS_ABOD)
PR_ABOD = np.array(PR_ABOD)
ABOD_res = np.mean(AS_ABOD, axis=0)
ABOD_prob = np.mean(PR_ABOD, axis=0)

In [ ]:
AS_KNN = []
PR_KNN = []
for t in tqdm(range(T)):
    X_corped = generate_corped_x(X)
    clf_KNN = KNN(contamination=0.1, n_neighbors=5, method='largest').fit(X_corped)
    Anomaly_score = clf_KNN.decision_scores_
    Anomaly_proba = clf_KNN.predict_proba(X_corped)
    AS_KNN.append(Anomaly_score)
    PR_KNN.append(Anomaly_proba)
AS_KNN = np.array(AS_KNN)
PR_KNN = np.array(PR_KNN)
KNN_res = np.mean(AS_KNN, axis=0)
KNN_prob = np.mean(PR_KNN, axis=0)

In [ ]:
doc_title_apple= "./data/company/title_apple.txt"
corpus = []
with open(doc_title_apple, "r" ) as f:
    for line in f:
        corpus.append(line)

In [ ]:
from pprint import pprint
pprint(corpus)

In [ ]:
for idx in ans:
    pprint(corpus[idx])

In [ ]:
PROB = LOF_prob + iforest_prob + KNN_prob

In [ ]:
PROB = PROB[:, 1]

In [ ]:
ano_idx = [i for i, val in enumerate(PROB) if val > 1.5]

In [ ]:
ano_idx

In [ ]:
for idx in ano_idx:
    pprint(corpus[idx])

In [ ]:
df_apple = pd.read_csv('./data/company/apple.csv')

In [ ]:
df_a = df_apple.iloc[ano_idx]

In [ ]:
df_a.loc[df_a["sentiment"] == -1]

In [ ]:
neg_idx = df_apple.index[df_apple['sentiment'] == -1].tolist()

In [ ]:
neg_idx

In [ ]:
res = np.intersect1d(ano_idx, neg_idx)

In [ ]:
for idx in res:
    pprint(corpus[idx])